In [1]:
pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 11.5 MB/s eta 0:00:00


In [2]:
import re
import nltk

# Use to handle punctuation
nltk.download('punkt')

import numpy as np
from nltk.tokenize import word_tokenize
import emoji

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


### Data Preparation

### Cleaning and tokenization

In [3]:
# Define corpus
corpus = 'Who ❤️ "word embeddings" in 2020? I do!!!"'

In [4]:
# Print original corpus
print(f"Corpus : {corpus}")

# Do the substitution
data = re.sub(r'[,!?;-]', '.', corpus)

# Print cleaned corpus
print(f"After cleaning punctuation : {data}")

Corpus : Who ❤️ "word embeddings" in 2020? I do!!!"
After cleaning punctuation : Who ❤️ "word embeddings" in 2020. I do..."


In [5]:
# Print the cleaned corpus
print(f"Initial string : {data}")

# Tokenize the cleaned corpus
data = nltk.word_tokenize(data)

# Print the tokenized version of the corpus
print(f"After tokenization : {data}")

Initial string : Who ❤️ "word embeddings" in 2020. I do..."
After tokenization : ['Who', '❤️', '``', 'word', 'embeddings', "''", 'in', '2020', '.', 'I', 'do', '...', "''"]


In [6]:
# Print the tokenized version of the corpus
print(f"Initial list of tokens : {data}")

# Filter tokenized corpus using list comprehension
data  = [
    ch.lower() for ch in data
    if ch.isalpha()
    or ch == '.'
    or bool(emoji.emoji_list(ch))
]

# Print the tokenized and filtered version of the corpus
print(f"After cleaning : {data}")

Initial list of tokens : ['Who', '❤️', '``', 'word', 'embeddings', "''", 'in', '2020', '.', 'I', 'do', '...', "''"]
After cleaning : ['who', '❤️', 'word', 'embeddings', 'in', '.', 'i', 'do']


In [7]:
def tokenize(corpus):
  data = re.sub(r'[,!?;-]+', '.', corpus)
  data = nltk.word_tokenize(data)
  data = [
      ch.lower() for ch in data
      if ch.isalpha()
      or ch == '.'
      or bool(emoji.emoji_list(ch))
  ]
  return data

In [8]:
# Define new corpus
corpus = "I am happy because I am learning"

# Print new corpus
print(f"Corpus : {corpus}")

# Save tokenized version of corpus into 'words' variable
words = tokenize(corpus)

# Print the tokenzied version of the corpus
print(f"Words (tokens) : {words}")

Corpus : I am happy because I am learning
Words (tokens) : ['i', 'am', 'happy', 'because', 'i', 'am', 'learning']


In [9]:
# Run this with any sentence
tokenize("My name is John. How are you?")

['my', 'name', 'is', 'john', '.', 'how', 'are', 'you', '.']

# Sliding window of words

In [10]:
# Define the 'get_windows' function
def get_windows(words, C):
  i = C
  while i < len(words) - C:
    center_word = words[i]
    context_words = words[(i - C) : i] + words[(i + 1) : (i + C + 1)]
    yield context_words, center_word
    i += 1

In [11]:
for x,y in get_windows(['i', 'am', 'happy', 'because', 'i', 'am', 'learning'], 2):
  print(f"{x}\t{y}")

['i', 'am', 'because', 'i']	happy
['am', 'happy', 'i', 'am']	because
['happy', 'because', 'am', 'learning']	i


In [12]:
# Print 'context_words' and 'center_word' for any sentence with a 'context half-size' of 1
for x, y in get_windows(tokenize("My name is John. How are you?"), 1):
    print(f'{x}\t{y}')

['my', 'is']	name
['name', 'john']	is
['is', '.']	john
['john', 'how']	.
['.', 'are']	how
['how', 'you']	are
['are', '.']	you


# Transforming words into vectors for the training set

In [13]:
def get_dict(data):
  words = sorted(list(set(data)))
  n = len(words)
  idx = 0

  # return these correctly
  word2Ind = {}
  Ind2word = {}
  for k in words:
    word2Ind[k] = idx
    Ind2word[idx] = k
    idx +=1

  return word2Ind, Ind2word

In [15]:
# Get "word2Ind" and "Ind2word" dictionaries for the tokenized corpus
word2Ind, Ind2word = get_dict(words)

In [16]:
# Print 'word2Ind' dictionary
word2Ind

{'am': 0, 'because': 1, 'happy': 2, 'i': 3, 'learning': 4}

In [17]:
# Print value for the key 'i' within word2Ind dictionary
print("Index of the word 'i' : ", word2Ind['i'])

Index of the word 'i' :  3


In [18]:
# Print 'Ind2word' dictionary
Ind2word

{0: 'am', 1: 'because', 2: 'happy', 3: 'i', 4: 'learning'}

In [19]:
# Print value for the key '2' within Ind2word dictionary
print("Word which has index 2 : ", Ind2word[2])

Word which has index 2 :  happy


In [20]:
# Save length of word2Ind dictionary into the 'V' variable
V = len(word2Ind)

# Print length of word2Ind dictionary
print("Size of vacabulary : ", V)

Size of vacabulary :  5


# Getting one-hot word vectors

In [21]:
# Save index of word 'happy' into the 'n' variable
n = word2Ind['happy']

# Print index of word 'happy'
n

2

In [22]:
# Create vector with the same length as the vocabulary, filled with zeros
center_word_vector = np.zeros(V)

# Print vector
center_word_vector

array([0., 0., 0., 0., 0.])

In [23]:
# Assert that the length of the vector is the same as the size of the vocabulary
len(center_word_vector) == V

True

In [25]:
# Replace element number 'n' with a 1
center_word_vector[n] = 1

# Print vector
center_word_vector

array([0., 0., 1., 0., 0.])

In [26]:
# Define the 'word_to_one_hot_vector' function
def word_to_one_hot_vector(word, word2Ind, V):
  one_hot_vector = np.zeros(V)
  one_hot_vector[word2Ind[word]] = 1
  return one_hot_vector

In [27]:
# Print output of 'word_to_one_hot_vector' function for word 'happy'
word_to_one_hot_vector('happy', word2Ind, V)

array([0., 0., 1., 0., 0.])

# Getting context word vectors

In [28]:
# Define list containing context words
context_words = ['i', 'am', 'because', 'i']